In [1]:
!pip install openai
import requests
import openai
import numpy as np
import json
import random
import pandas as pd
import pickle
from pathlib import Path
from PIL import Image
from typing import List, Tuple
from io import BytesIO

import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 3.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 57.8 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=b6b3ab9df5d188fd2f7500fe23d7e3afef93a833eb6a5550cef99aed2747b7fd
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


In [11]:
DATA_DIR = Path("gdrive/MyDrive/data/")


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
labels = pd.read_csv("labels.csv")

In [4]:

def query_dalle(prompt, n=1):
  return openai.Image.create(
    prompt=prompt,
    n=n,
    size="256x256"
  )

def read_json(path) -> dict:
    with open(path) as f:
        return json.load(f)
      

config = read_json("openai_config.json")
openai.api_key = config["key"]



In [9]:
MONK = {
    "monk_1": "very pale",
    "monk_2": "pale",
    "monk_3": "light",
    "monk_4": "olive",
    "monk_5": "light brown",
    "monk_6": "brown",
    "monk_7": "dark brown",
    "monk_8": "dark",
    "monk_9": "black",
    "monk_10": "very black"
}

def create_prompt(age, gender, skin_tone) -> str: 
  age_min, age_max = age.split("_")
  new_age = random.randint(a=int(age_min), b=int(age_max))
  monk = MONK[skin_tone] 
  base_prompt =  "a face photo of a XXX year old, YYY-skinned ZZZ, photo-realistic"
  return base_prompt.replace("XXX", str(new_age)).replace("YYY", monk).replace("ZZZ", gender)



In [10]:
import itertools

combis = list(itertools.product(labels["age"].dropna().unique(), ["male", "female"], MONK.keys()))
prompts = {(age, gender, skin_tone): create_prompt(age=age, gender=gender, skin_tone=skin_tone) for age, gender, skin_tone in combis}

In [25]:
from tqdm import tqdm
import time

responses = {cats: [] for cats in prompts}
for cats, prompt in tqdm(prompts.items()):
  try:
    response = query_dalle(prompt)
    responses[cats].append(response["data"])
  except openai.APIError:
    print("caught an error!")
    time.sleep(10)
    try:
      response = query_dalle(prompt)
      responses[cats].append(response["data"])
    except openai.APIError as e:
      print("another one")
      raise e 
  time.sleep(5)  

100%|██████████| 80/80 [14:15<00:00, 10.70s/it]


In [35]:
def read_img(path, img_size: Tuple[int, int]=(224,224)) -> Image.Image:
    return Image.open(path).convert("RGB").resize(img_size)

def read_img_url(url: str) -> Image.Image:
    response = requests.get(url)
    return read_img(BytesIO(response.content))

imgs = {cat: read_img_url(respons[0][0]["url"]) for cat, respons in responses.items()}

In [43]:
img_array = np.array([np.array(img) for img in imgs.values()])
img_array.shape


(80, 224, 224, 3)

In [46]:
import pickle 
from pathlib import Path


with open(DATA_DIR / "dalle_img_dict.pkl", "wb") as f:
  pickle.dump(imgs, f)

In [3]:
def create_vgg16():
    # download VGG-16 with fully connected layers
    vgg = tf.keras.applications.vgg16.VGG16(
        include_top=True,
        weights="imagenet",
        input_shape=(224, 224, 3),
        pooling=None,
        classes=1000,
        classifier_activation="softmax",
    )
    # remove the classification layer
    new_model = tf.keras.models.Sequential()
    for layer in vgg.layers[:-1]:
        new_model.add(layer)
    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    x = tf.keras.applications.vgg16.preprocess_input(inputs)
    outputs = new_model(x)
    final_vgg = tf.keras.Model(inputs, outputs)
    return final_vgg

In [5]:
vgg16 = create_vgg16()

553467096/553467096 [==============================] - 24s 0us/step


In [12]:
def read_pickle(path: Path): 
  with open(path, "rb") as f:
    return pickle.load(f)

img_dict = read_pickle(DATA_DIR / "dalle_img_dict.pkl")

In [13]:
img_arr = np.array([np.array(img) for img in img_dict.values()])

In [15]:
dalle_preds = vgg16.predict(img_arr)
assert dalle_preds.shape[0] == 80

3/3 [==============================] - 0s 173ms/step


In [86]:
from scipy.spatial import KDTree
from collections import Counter
from typing import Dict, Tuple, List

def most_common(arr) -> int:
  counts = np.bincount(arr.astype(int))
  return np.argmax(counts)


def map_to_int(x: List[str]) -> Tuple[np.array, Dict[int, str]]:
    """Map a list of strings to a list of ints and a dict mapping ints to strings."""
    mapping = {s: i for i, s in enumerate(set(x))}
    return np.array([mapping[s] for s in x]), mapping

def invert_dict(d: dict) -> dict:
  return {v: k for k, v in d.items()}

class DalleKNN:
  def __init__(self, labels: List[Tuple[str, str, str]]):
    self.labels = labels
    self.ages, self.age_map = map_to_int([x[0] for x in self.labels])
    self.genders, self.gender_map =  map_to_int([x[1] for x in self.labels])
    self.skin_tones, self.skin_tone_map =  map_to_int([x[2] for x in self.labels])
    assert len(np.unique(self.skin_tones)) == 10, "mismatch!"
    self.age_map = invert_dict(self.age_map)
    self.gender_map = invert_dict(self.gender_map)
    self.skin_tone_map = invert_dict(self.skin_tone_map)
    
  def fit(self, dalle_preds: np.ndarray):
    self.kdtree = KDTree(dalle_preds)
  
  def find_match(self, new_img, k=3) -> Tuple[str, str, str]:
    dist, idx = self.kdtree.query(new_img, k=k)
    age_pred = most_common(self.ages[idx])
    gender_pred = most_common(self.genders[idx])
    skin_tone_pred = most_common(self.skin_tones[idx])
    return self.age_map[age_pred], self.gender_map[gender_pred], self.skin_tone_map[skin_tone_pred]
  


In [31]:
test_preds = np.load(DATA_DIR / "vgg_test_preds.npy")

In [113]:
cat_list = [cat for cat in img_dict]

test_knn = DalleKNN(cat_list)
test_knn.fit(dalle_preds)
test_knn.find_match(dalle_preds[0])

with open(DATA_DIR / "oii_unsupervised.pkl", "wb") as f:
  pickle.dump(test_knn, f)

In [115]:
new_knn = read_pickle(DATA_DIR / "oii_unsupervised.pkl")

('0_17', 'male', 'monk_1')

In [79]:
test_labels = pd.read_csv(DATA_DIR / "test_labels.csv", index_col=0)
test_imgs = np.load(DATA_DIR / "vgg_test_preds.npy")


In [97]:
preds = {"age": [], "gender": [], "skin_tone": []}

for (i, row), test_img in zip(test_labels.iterrows(), test_imgs):
  agepred, genderpred, skinpred = test_knn.find_match(test_img)
  preds["age"].append(agepred)  
  preds["gender"].append(genderpred)  
  preds["skin_tone"].append(skinpred)  


In [104]:
from sklearn.metrics import confusion_matrix, accuracy_score
# calculate disparity
def disparity_score(ytrue, ypred):
    cm = confusion_matrix(ytrue,ypred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    all_acc = list(cm.diagonal())
    return max(all_acc) - min(all_acc)

def evaluate_dict(test_df, pred_dict, score_func): 
  return {category: score_func(test_df[category], prediction) for category, prediction in pred_dict.items()}


In [106]:
results = {}
results["accuracy"] = evaluate_dict(test_labels, pred_dict=preds, score_func=accuracy_score)
results["disparity"] = evaluate_dict(test_labels, pred_dict=preds, score_func=disparity_score)
results

{'accuracy': {'age': 0.22966666666666666,
  'gender': 0.6896666666666667,
  'skin_tone': 0.112},
 'disparity': {'age': 0.47205671364672125,
  'gender': 0.07009798793458166,
  'skin_tone': 0.3431952662721893}}

In [110]:
def getScore(results):
    acc = results['accuracy']
    disp = results['disparity']
    ad = 2*acc['gender']*(1-disp['gender']) + 4*acc['age']*(1-disp['age']**2) + 10*acc['skin_tone']*(1-disp['skin_tone']**5)
    return ad

title = "OII Gang Unsupervised"

submission = {
    'submission_name': title,
    'score': getScore(results),
    'metrics': results
}
submission

{'submission_name': 'OII Gang Unsupervised',
 'score': 3.1112656662779923,
 'metrics': {'accuracy': {'age': 0.22966666666666666,
   'gender': 0.6896666666666667,
   'skin_tone': 0.112},
  'disparity': {'age': 0.47205671364672125,
   'gender': 0.07009798793458166,
   'skin_tone': 0.3431952662721893}}}

In [112]:
with open(DATA_DIR / "unsupervised_oii_results.json", "w") as f:
  json.dump(submission, f)